# Data minining with Python

#### Обычно очень много винимания в процессе обученя анализу данных уделяется алгоритмам и методам обработки данных и намного меньше внимания уделяется тому эти данные взять. При этом: <br>
- Без самих данных навряд ли что-то получится; <br>
- От качества данных зависит результат работы любого алогоритма; <br>
- Часто работают лучше те модели которые обучались на более качественных или большем объеме данных.

### Где брать данные?

Конечно, данные вам могут предоставить на работе или заказчик. Вы можете работать в одной из компаний где данных больше чем предостаточно. А что если их не достаточно или в них нет того что вам нужно.


### В этом случае лучше всего действовать в следующей последовательности:

- Найти готовый датасет;
- Собрать данные самому:
   - для этого можно воспользоваться API;
   - собрать даные со страниц сайтов.
   
Существует несколько инструментов для парсинга данных самостоятельно но чаще всего используют:
- библиотеку requests + библиотеку beautiful soup
- фремйворк Scrappy 

Оба варианта взаимоисключающие и необходимость использовать тот или иной зависит от случая, подробнее об этом будет ниже.


### Поиск готового датасета.
Логичнее всего начать с поиска готового датасета. Существует много сайтов где можно найти уже готовые данные, например, очень много данных есть на платформе <a href="https://www.kaggle.com/datasets">Kaggle<a/>. Менее популярная платформа, но не менее интересная <a href="http://academictorrents.com/browse.php">academictorrents.com </a>, советую проверить. <br>
Существуют и еще менее тривиальные источники данных о существовании которых иногда трудно догодаться. Так например, есть Google Big Querry на котором сейчас выложено около 40 публичных датасетов, один из них это все записи форума reddit. Казалось бы зачем нам форум такой форум? На самом деле применений может быть очень много, например, по его данным легко обучить классификатор текстов на разные категории, а категорий на форуме очень много.  <br>
    
Интерфейс у Google Big Querry это SQL, что очень удобно и можно сразу вытащить то что нужно. Попробовать можно здесь: <a href="https://bigquery.cloud.google.com/results/bigdatagame-200413:US.bquijob_609efa7_16871e6cd5e?pli=1">Google Big Querry</a> <br>
Пример запроса который достает 1500 комментариве из раздела "Art" c лайками больше 5ти. <br>
    
<strong><i>SELECT body, score FROM <br>
[fh-bigquery:reddit_comments.2017_05], <br>
WHERE subreddit LIKE 'Art' AND score > 5 <br>
LIMIT 15000 <br></i></strong>

### Самостоятельный сбор данных с помощью API.
Ок, если не удалось найти готовый датасет, то возможно они есть в отрытом доступе на каком-то сайте или платформе через общедоступный API. <br>
 <br>
#### Работа с API выглядит следующим образом. 
Есть два метода (на самом деле их больше) post и get с помощью которых мы можем обратиться к сайту или платформе. С помощью метода get мы можем можем сделать запрос ( напрмер когда мы открываем сайт по ссылке то это get), а с помощью метода post мы можем передать какеи-то значения клиенту.<br>
Так например, сайт HH.ru предоставляет свой API и по следующему запросу мы можем получить список вакансий определенной категории 1.221 (цифры обозначающие вакансии это их внутренняя кухня, тут без лишних деталей). <br>
https://api.hh.ru/vacancies?specialization=1.221 <br>
В ответ на странице вы увидите скорее всего Json фаил, который очень легко читать и с которым приятно работать. <br>
Теперь нам нужно понять что с этим делать дальше.

### Работа с библиотекой requests
Собственно для работы с post и get запросами нам и понадобиться библиотека requests.

Для установки библиотеки используйте следующую комманду в терминале (если на Linux)

pip install requests

Документация по библиотеке requests: http://docs.python-requests.org/en/master/

### Посмотрим на примере сайта HH.ru как работать с библиотекой и как можно получить какие-то данные с помощью API

In [1]:
import requests
url1 = 'https://api.hh.ru/vacancies?specialization=1.221'
url2 = 'https://api.hh.ru/vacancies/1'
resp = requests.get(url2)
print(resp)
resp = requests.get(url1)
print(resp)

<Response [404]>
<Response [200]>


#### Мы видим что для одной из ссылок клиент вернул статус 404, а для другой 200.

Это код ответа от клиента, код 200 обозначает что все ок, а код 404 обозначает Not found. Полный списко кодов состояния HTTP можно посмотреть <a href="https://ru.wikipedia.org/wiki/Список_кодов_состояния_HTTPhttps://ru.wikipedia.org/wiki/Список_кодов_состояния_HTTP"> вот здесь. </a> В дальнейшем, в рамках этого туториала нас интересует только код 200. Но на практике полезно знать и другие. <br>
И так, для одной из ссылок мы получили 200, все ок, двигаемся дальше.

In [2]:
url=url1+'&page=1&per_page100'
resp = requests.get(url)
print(resp)

<Response [200]>


'&page=1&per_page100'  - это опять внутренняя кухня API HH.ru, у всех платформ она разная. Здесь мы говорим что мы загружаем первую страницу и 100 результатов на странице. Почему нельзя сделать больше или подругому за раз зависит от того как это решили разработчики API. <br>
Но можно получить больше результатов следующим образом.


In [3]:
searches = []
for page in range(20):
    searches.append(
        requests.get(url1 + '&page={}&per_page=100'.format(page)).json()
    )

In [4]:
#Можно открыть первую ссылку в нашем дасатае и посмореть что мы собрали.
searches[0]['items'][0]['url']

'https://api.hh.ru/vacancies/29693966?host=hh.ru'

In [5]:
vacs = [x['items'] for x in searches]
vacs = sum(list(vacs),[])
vacs[0]

{'id': '29693966',
 'premium': False,
 'name': 'Senior Android developer',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '26', 'name': 'Воронеж', 'url': 'https://api.hh.ru/areas/26'},
 'salary': {'from': 120000, 'to': 150000, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'response_url': None,
 'sort_point_distance': None,
 'employer': {'id': '1321508',
  'name': 'AdCombo.com',
  'url': 'https://api.hh.ru/employers/1321508',
  'alternate_url': 'https://hh.ru/employer/1321508',
  'logo_urls': {'original': 'https://hhcdn.ru/employer-logo-original/259233.png',
   '90': 'https://hhcdn.ru/employer-logo/1479462.png',
   '240': 'https://hhcdn.ru/employer-logo/1479463.png'},
  'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=1321508',
  'trusted': True},
 'published_at': '2019-04-13T01:24:15+0300',
 'created_at': '2019-04-13T01:24:15+0300',
 'archived': False,
 'apply_alternate_url': 'h

In [6]:
#Посморим на первые 5 названий вакансий
titles = [item['name'].lower() for item in vacs]
titles[:5]

['senior android developer',
 'senior ios developer',
 'web-дизайнер',
 'junior backend php developer',
 'middle backend php developer']

In [7]:
#А теперь очень просто найдем самое часто встречаемое название вакансии

max(set(titles), key=titles.count)

'программист 1с'

Использовать API можно не только для текстовой информации, хотя чаще всего для нее. Вот пример работы c API Flickr https://gist.github.com/diff7/627a306d6d8e4ecd0162808deae7d244 для получения картинок. Для использования нужно импортировать API key на сайте и установить пакеты, которые импортируются в начале скрипта, если они еще не стоят. <br>


А если вам вдруг понравилось работать с апишками то вот здесь есть очень хороший проект с кучей всяких разных API со всего интернета.
<a href="https://public-apis.xyz/">public-apis.xyz</a>

###  Beautiful Soup
Документация : https://www.crummy.com/software/BeautifulSoup/bs4/doc/ <br>

Чаще всего через API можно получить намного больше интересной и качественной информации, убедиться в этом можно опять же зайдя на сайт public-apis.xyz там есть и статистика из онлайн игр отзывы по фильмам и много всего. Но  иногда сайты не предоставляют API, но данные на них все таки есть. (Мы не обсуждаем можно ли их использовать или нет, скорее всего об этом написано на самом сайте) Тогда приходиться дейстовать подругому. <br>
Посмотрим на примере сайта http://quotes.toscrape.com который был создан для как раз для практики данных навыков. <br>

In [8]:
link='http://quotes.toscrape.com'
page=requests.get(link)
if page.status_code == 200:  
    print(page.content[700:1100]) #Посмотрим на то что мы получили между 800 и 1100 набором символов

b'"col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">\xe2\x80\x9cThe world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.\xe2\x80\x9d</span>\n        <span>by <small class="author" itemprop="author">Albert Einstein</small>\n        <a href="/author/Albert-Einstein">(about)</a>\n      '


Как видно здесь присутствует какой - то текст, но ответ от страницы нам пришел так как видит ее браузер, т.е. с HTML тегами, сss и возможно JavaScript. Собственно что бы очистить страницу от HTML и всего прочего нам и нужна библиотека Beautiful Soup.

### Чуть - чуть про HTML, если вы в курсе то пропустите.

<!DOCTYPE html>  
<html>  
    <head  style="color:red">Это простой пример HTML разметки используемый для форматирования текста.</head>
    <div> Если нажаит на <strong>этот текст</strong> два раза, то вы увидите его так как его видит браузер.</div>
    <body>
        <h1><span class="cm-string"> Заголовок &lt;h1&gt; </span></h1>
        <p>Параграф &lt;p&gt; </p>
        <a href="www.nowhere.space">Ссылка в никуда</a>
        <div>блок &lt;div&gt; </div>
        <h2  style="color:red" > И конечно картинка: </h2>
        <img src="https://cs6.pikabu.ru/post_img/big/2015/06/18/3/1434596941_632146314.jpg">
        <p style="color:blue;">Для того что бы просмотреть страницу в HTML можно нажать правой кнопкой мыши и выбрать "inspect" в появившимся меню.</p>
        <p>В общем, функционал зависит от браузера, но в целом должно появится что-то вроде этого. Можете попробовать с сайтом
            <a href="http://quotes.toscrape.com"> http://quotes.toscrape.com </a></p>
        <img src="https://mydatacareer.com/wp-content/uploads/2017/05/scraping3.png">
    </body>
</html>

#### Теперь мы знаем каким элементом страницы является тот или иной текст и сможем легко его извлечь с помощью библиотеки Beautiful Soup

Для установки библиотеки используйте следующую комманду в терминале (если на Linux)

pip install beautifulsoup4

In [9]:
import bs4 as bs

link='http://quotes.toscrape.com'
page=requests.get(link)
if page.status_code == 200:  
    soup = bs.BeautifulSoup(page.content, 'html.parser')
    
soup.find_all('span', attrs={'class': 'text'}) #Как видно из картинки выше, наша цитата находится внутри тега span c классом text

[<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>,
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>,
 <span class="text" itemprop="text">“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”</span>,
 <span class="text" itemprop="text">“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”</span>,
 <span class="text" itemprop="text">“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”</span>,
 <span class="text" itemprop="text">“Try not to become a man of success. Rather become a man of value.”</span>,
 <span class="text" itemprop="text">“It is better to be hated for what you are than to be loved for what you are not.

In [10]:
tags=soup.find_all('span', attrs={'class': 'text'})
tags[0].text

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [11]:
[item.text for item in tags]

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

### Недостатки такого метода

С помощью комбинации Beautiful Soup + Requests можно получить данные с большого числа сайтов. Но существует ряд недостаткво.

- Достаточно долго, так как процесс не параллелиться;
- Для каждого нового сайта приходится писать все заново;
- Некоторые сайты блокируют возможность парсинга, а некоторые используют JavaScript для отрисовки вместо HTML, тогда связки Beautiful Soup + Requests будет недостаточно.

### Scrapy
Scrapy это уже не библиотека, а полноценный фреймворк для парсинга данных с сайтов с кучей настроек и шикарным функционалом.
Основным недостатком Scrapy является достаточно высокий порог входа, придеться потратить некоторое время что бы разобраться во всех деталях. По дизайну фреймворк часто сравнивают с Django.

Scrapy лучше использовать в следующих случаях:
- Кодга важна скорость скачивания;
- Когда данные парсятся с большого числа однотипных сайтов;
- Когда предстоит парсить данные с одого или нескольких сайтов на регулярной основе;
- Когда нужно спарсить очень большой сайт.

### Работа со Scrapy на примере сайте quotes.toscrape.com

В командной строке

pip install scrapy

дальнейшие команды нужно делать в системном питоне

In [ ]:
#запустить Scrapy проект 
scrapy startproject proj

#в директории проекта должна появится папка spiders
cd scrapy/proj/proj/spiders 

#cоздайте в директории spiders фаил quotes_spider.py
nano quotes_spider.py

In [ ]:
#содержимым файла будет соледующий код

# -*- coding: utf-8 -*-
import scrapy

class QuotesSpiderSpider(scrapy.Spider):
    name = 'quotes_spider' #название паука
    allowed_domains = ['quotes.toscrape.com'] 
    start_urls = ['http://quotes.toscrape.com/'] #ссылка с которой начинается парсинг

    def parse(self, response):
        #для выделения HTML тегов или сss декораторов Scrapy использует Xpath
        #по сути здесь мы передаем теже параметры которые передавали в Beautiful Soup
        quotes = response.xpath("//div[@class='quote']//span[@class='text']/text()").extract()
        yield {'quotes': quotes}

In [ ]:
#После этого можно запустить Scrapy со следующей командой
scrapy crawl quotes_spider -o quotes.csv

#После исполнения в вашей директории должен появнится фаил quotes.csv 
#А ниже представлен примерный лог выполненной работы вашего паука

In [ ]:
{'downloader/request_bytes': 446,
 'downloader/request_count': 2
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 2701,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 1,
 'downloader/response_status_count/404': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2019, 1, 22, 21, 36, 6, 198926),
 'item_scraped_count': 1,
 'log_count/DEBUG': 4,
 'log_count/INFO': 8,
 'memusage/max': 53608448,
 'memusage/startup': 53608448,
 'response_received_count': 2,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2019, 1, 22, 21, 36, 5, 559859)}
2019-01-23 00:36:06 [scrapy.core.engine] INFO: Spider closed (finished)

Еще один момент который хочеться упомямунть это то что Scrapy имеет удобный shell интерфейс для настройки пауков. По факту это обычный Ipython, но с набором внутренних команд Scrapy.

Shell запускается командой scrapy shell.